# Problem Statement

Lending Club wants to understand the driving factors (or driver variables) behind loan default, i.e. the variables which are strong indicators of default.  The company can utilise this knowledge for its portfolio and risk assessment.  
Two types of risks are associated with the bank’s decision:  

* If the applicant is likely to repay the loan, then not approving the loan results in a loss of business to the company  

* If the applicant is not likely to repay the loan, i.e. he/she is likely to default, then approving the loan may lead to a financial loss for the company  
  
The provided data contains the information about past loan applicants and whether they ‘defaulted’ or not. The aim is to identify patterns which indicate if a person is likely to default, which may be used for taking actions such as denying the loan, reducing the amount of loan, lending (to risky applicants) at a higher interest rate, etc.  
  
When a person applies for a loan, there are two types of decisions that could be taken by the company:  

1. Loan accepted: If the company approves the loan, there are 3 possible scenarios described below:

 * Fully paid: Applicant has fully paid the loan (the principal and the interest rate)

 * Current: Applicant is in the process of paying the instalments, i.e. the tenure of the loan is not yet completed. These candidates are not labelled as 'defaulted'.

 * Charged-off: Applicant has not paid the instalments in due time for a long period of time, i.e. he/she has defaulted on the loan 

2. Loan rejected: The company had rejected the loan (because the candidate does not meet their requirements etc.). This data is not part of provided data set

# Data Understanding

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import missingno as msno
import seaborn as sns
import numpy as np
from statsmodels.graphics.mosaicplot import mosaic
import scipy.stats as ss
import numpy as np
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.libqsturng import psturng

loading the data from loan.csv file. While doing so low_memory is set to False to avoide mixed data type warning

In [ ]:
# Load the loan data, low_memory set to False to avoide mixed data type warning
loan_all_df = pd.read_csv("loan.csv", low_memory=False)
loan_all_df.shape

Let's see inspect the columns in the data frame

In [ ]:
loan_all_df.head()

In [ ]:
loan_all_df.info()

As all of our analysis will revolve around loan_status (our target variable), lets see the different values and their proportions in the data.

In [ ]:
# set figure size
sns.set(rc={'figure.figsize':(12,10)})
# see proportion of loan status our target variable
sns.countplot(loan_all_df.loan_status)

As our goal would be to find differentiating factors between customers who would fully pay vs customer who would default, our observation will come from these two sets of customers in the historic data.  
Therefore we can ignore the data where the loan is currently ongoing. We would not know whether these will be fully paid or defaulted.

In [ ]:
# Keep only the rows where loan_status is not Current
loan_df = loan_all_df.loc[loan_all_df.loan_status != 'Current']
loan_df.shape

Let's see what is the proportion of charged off customers with respect to Fully paid in the data set 

In [ ]:
print(loan_df.groupby("loan_status")['loan_status'].agg(['count'])/len(loan_df))
      

That Fully paid is around 85% and Charged off is around 15%

# Data Cleaning and Manipulation

## Data Sanity Checks

Lets first do a round of sanity check to remove columns that does not have any information or does not have any variance

In [ ]:
# Find out percentage of missing values in each column
columns = loan_df.columns
percent_missing = loan_df.isnull().sum() * 100 / len(loan_df)
missing_value_df = pd.DataFrame({'column_name': columns,
                                 'percent_missing': percent_missing})
missing_value_df.sort_values('percent_missing', ascending = False, inplace=True)

# list the columns that have 100% missing values
missing_value_df[missing_value_df.percent_missing == 100]

All those above columns can be deleted as they don't have any information and is not going to help us in the analysis anyways. If business think that any of these columns may have impact to find difference between fully paid and charged off then we are not looking at right sample. Business should be advised to provide right sample set of data.

In [ ]:
# drop columns that have 100% mising values
loan_df.drop(columns=list(missing_value_df.loc[missing_value_df.percent_missing == 100,'column_name']),inplace = True)
loan_df.shape

Lets find out numeric columns where there is no variance, i.e. it contains same value in all its rows.

In [ ]:
# find out columns that have 0 standard deviation
loan_df.std()[loan_df.std() == 0]

These columns are not going to help us either

In [ ]:
# dropping column with standard deviation 0
loan_df.drop(loan_df.std()[loan_df.std() == 0.0].index.values, axis=1,inplace = True)
loan_df.shape

Lets find out non-numeric columns containing single values only.

In [ ]:
# checking whether there are columns having single values. Standard deviation check will not work for non-numeric columns
uniques = loan_df.apply(lambda x: x.nunique())
uniques[uniques==1]

The above non-numeric columns contain single values. Thus not helpful either.

In [ ]:
# dropping columns having single values
loan_df = loan_df.drop(uniques[uniques==1].index, axis=1)
loan_df.shape

After understanding the data data definitions from data dictionary we see that following fields will not be available for new customer while applying for loan. Therefore these features cannot be used for finding features.  
* `last_credit_pull_d` - The most recent month LC pulled credit for this loan
* `last_pymnt_amnt` - Last total payment amount received
* `collection_recovery_fee` - post charge off collection fee
* `delinq_2yrs` - The number of 30+ days past-due incidences of delinquency in the borrower's credit file for the past 2 years
* `mths_since_last_delinq` - The number of months since the borrower's last delinquency.
* `last_pymnt_d` - Last month payment was received
* `recoveries`  - post charge off gross recovery
* `total_pymnt` - Payments received to date for total amount funded
* `total_pymnt_inv`- Payments received to date for portion of total amount funded by investors
* `total_rec_int` - Interest received to date
* `total_rec_late_fee` - Late fees received to date
* `total_rec_prncp` - Principal received to date

In [ ]:
# Drop columns that will not be available at the time of aproving loan
columns_to_remove = ['last_credit_pull_d',
                     'last_pymnt_amnt',
                    'collection_recovery_fee',
                     'delinq_2yrs',
                     'mths_since_last_delinq',
                     'last_pymnt_d',
                     'recoveries',
                     'total_pymnt',
                     'total_pymnt_inv',
                     'total_rec_int',
                     'total_rec_late_fee',
                     'total_rec_prncp'
                    ]
loan_df.drop(columns_to_remove , axis = 1 , inplace = True)
loan_df.shape

Lets now find out the columns that have very high unique values

In [ ]:
# Check potential unique value fields 
df_colunique = pd.DataFrame(loan_df.nunique()/loan_df.count())
# Columns having all unique values excludeing Nulls
df_colunique.loc[df_colunique[0] >= 0.99]

As we can see these are mostly id columns or free text columns. Therefore we'll not be able to get any pattern form these columns

In [ ]:
# drop the columns where each row has unique values
loan_df.drop(df_colunique.loc[df_colunique[0] >= 0.99].index, axis=1, inplace= True)
loan_df.shape

### Missing Value treatments

Lets see the missing values in the data

In [ ]:
# visualize missing data
msno.matrix(loan_df)

The above and below graph shows that there missing values the following 4 columns:
* `emp_length`
* `emp_title`
* `title`
* `revol_util`
* `mths_since_last_record`
* `pub_rec_bankruptcies`  
Amongst these only `mths_since_last_record` has huge number of missing records

In [ ]:
msno.bar(loan_df)

The missingno correlation heatmap measures nullity correlation: how strongly the presence or absence of one variable affects the presence of another.  
The below heat map shows that `emp_length` and `emp_title` are missing together in 60% cases and `mths_since_last_record` and `pub_rec_bankruptcies` are missing together 50% times.

In [ ]:
msno.heatmap(loan_df)

In [ ]:
# Impute missing values
values = {'emp_length': 'NA', 'emp_title': 'NA', 'title': 'NA',\
          'revol_util': '0%', 'mths_since_last_record': -1, 'pub_rec_bankruptcies':0}
loan_df.fillna(value=values, inplace = True)
msno.bar(loan_df)

### Data Manipulations

In general we'll first delete and leading and training spaces

In [ ]:
# Strip all columns for leading and lagging spaces
loan_df = loan_df.applymap(lambda s : s.strip() if type(s) is str else s)

We observe that the term field has month as suffix, so we trim that and convert to number

In [ ]:
# Remove string months from term 
loan_df.term = loan_df.term.apply(lambda s : s[0:s.index(' ')] if ' ' in s else s)
loan_df.term = pd.to_numeric(loan_df.term)
loan_df.term.head()

We observe that int_rate and revol_util field has % as suffix, we remove that and convert to number

In [ ]:
# Remove % sign from interest rate 
loan_df.int_rate = loan_df.int_rate.apply(lambda s : s[0:s.index('%')] if '%' in s else s)
loan_df.int_rate = pd.to_numeric(loan_df.int_rate)
loan_df.int_rate.head()

In [ ]:
# Remove % sign from revol_util
loan_df.revol_util = loan_df.revol_util.apply(lambda s : s[0:s.index('%')] if '%' in s else s)
loan_df.revol_util = pd.to_numeric(loan_df.revol_util)
loan_df.revol_util.head()

In [ ]:
loan_df_cat = loan_df.select_dtypes(['object'])
# Check potential unique value fields 
loan_df_cat_colunique = pd.DataFrame(loan_df_cat.nunique()/loan_df_cat.count())
# Columns having all unique values excludeing Nulls
loan_df_cat_colunique

We found that following columns either have very high variance or does not add much information
* `title`: very high variance
* `emp_title`: very high variance
* `zip_code`: Not so high variance but lots of values. Addr_state can cover for this variable.

In [ ]:
# Drop columns that will can not have any impact
columns_to_remove = ['title' , # very high variance
                     'emp_title', # very high variance
                     'zip_code' # Not so high variance but lots of values where addr_state can cover
                    ]
loan_df.drop(columns_to_remove , axis = 1 , inplace = True)
loan_df.shape

Lets see if we have any duplicate rows.

In [ ]:
# checking whether there are duplicate rows
loan_df.duplicated().sum()

In [ ]:
loan_df.info()

We'll progress our analysis on the above 26 variables and 38577 rows

# Data analysis

## Custom Functions

### Univariate and segmented univariate analysis on categorical variables

The following function is created to do univariate and segmented univariate analysis of categorical variables. This function will accept a data frame having subject column and loan_status column will provide following graphs.
* rank frequency plot
* bar plot
* grouped bar plot
* Mosaic plot

In [ ]:
# Perform Univariate Analysis

# Write Function to perform univariate analysis on categorical variables 

def uni_analysis_cat (x , summary = 0, pct_label = True):
    print(x.iloc[:,0].describe())
    # preparing data for label in mosaic plot
    count_table = x.groupby(list(x.columns)).size()#.reset_index(name='counts')
    pct_table = count_table.groupby(level=0).apply(lambda x:
                                                 round(x / float(x.sum()),2))\
                                    .unstack(fill_value=0)
                                
    pct_table.index = pct_table.index.map(str)
    pct_table = pct_table.stack()
   
    sns.set(rc={'figure.figsize':(15,6)})
    # rank frequency plot
    ax = x.iloc[:,0].value_counts().plot(kind='barh')
    ax.set(xlabel="count", ylabel=list(x.columns)[0])
    plt.show()
    f, axes = plt.subplots(1, 2)
       
    if(~summary):
        print('****************************************************************************************************')
        # bar plot
        sns.countplot( x = x.iloc[:,0], data = x , ax = axes[0])
        
        # Grouped bar plot
        sns.countplot( x = x.iloc[:,0], hue = x.iloc[:,1] , data = x , ax = axes[1] )
        
        # change label direction
        for ax in f.axes:
            plt.sca(ax)
            plt.xticks(rotation=90)
        plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
        
        # create proportion labelizer for mosaic plot
        if pct_label:
            labelizer = lambda k: pct_table.loc[pd.IndexSlice[k[0]],pd.IndexSlice[k[1]],:].values[0]
        else:
            labelizer = lambda k: ''
        
        # mosaic plot
        mosaic(x.sort_values(list(x.columns)[0]),
               list(x.columns),
               label_rotation = [90,0], 
               labelizer=labelizer)

### Univariate and segmented univariate analysis for quantitative variable 

The following function is created to do univariate and segmented univariate analysis of numeric variables. This function will accept a data frame having subject column and loan_status column will provide following graphs.
* box plot
* segmented box plot
* segmented distribution plot

In [ ]:
# Univariate analysis for quantitative variable 


def uni_analysis_num (x , type = 'r', log_scale=False):
    sns.set(rc={'figure.figsize':(15,6)})
    f, axes = plt.subplots(1, 3)
    
    # print column description
    print(round(x.iloc[:,0].describe(),2))

    print('****************************************************************************************************')
    # for each unique value in 2nd column
    for value in list(x.iloc[:,1].unique()):
        a = x.loc[x.iloc[:,1] == value,list(x.columns)[0]]
        a.rename(index = value, inplace = True)
        # kde plot for 1 distribution
        p1=sns.kdeplot(a, shade=True)
    if log_scale: p1.set_xscale('log')
    p1.set_xlabel(list(x.columns)[0])
    p1.set_ylabel('Probability')
    
    # mean indicator marker
    meanpointprops = dict(marker='D', markeredgecolor='black',
              markerfacecolor='firebrick')
    # box plot
    bp = sns.boxplot( y = x.iloc[:,0], data = x , ax = axes[0],color = 'g',
                     showmeans=True, meanprops=meanpointprops, meanline=False)
    if log_scale: bp.set_yscale('log')

    # segmented box plot
    vp = sns.boxplot( x = x.iloc[:,1] , y = x.iloc[:,0] , data = x , ax = axes[1],\
                     showmeans=True, meanprops=meanpointprops, meanline=False)
    #vp = sns.violinplot( x = x.iloc[:,1] , y = x.iloc[:,0] , data = x , ax = axes[1], \
    #               vert = True , inner = 'quartile' ,scale_hue = True )
    if log_scale: vp.set_yscale('log')


### Bubble Plot to visualise date dependency

In [ ]:
def bubble_plot(x , y , z ,percent):
    plt.rcParams.update({'font.size': 12})

    plt.figure(figsize=(10, 6))
    
    plt.scatter(x = x
                , y = y , 
                s = percent * 100,
                c = 'r',
               alpha = 0.8)

    
    for i in range(0,len(x)):
        plt.text(x[i] ,y[i], s= percent[i].round(2) , ha = 'left', va = 'center', fontsize = 12)

    plt.show()

### Bivariate Analysis for Quantitative variables

In [ ]:
# Bivariate analysis : Common function to perform correlation within data frame 

def bi_analysis_num(df):
    sns.set(rc={'figure.figsize':(15,6)})
    f, axes = plt.subplots(1, 2)
    
    #sns.heatmap(df.corr(), fmt='0.2f', annot=True , cmap = 'Greens')
    dfc = df.loc[df.loan_status == 'Charged Off']
    dff = df.loc[df.loan_status == 'Fully Paid']
    
    print('The correlation matrix is as below')
    print(df.corr())
    print('**********************************************************************************')
    
    sns.kdeplot(dfc.iloc[:,0] , dfc.iloc[:,1] ,  cmap="Reds", shade=True, shade_lowest=False , ax = axes[0])
    sns.kdeplot(dff.iloc[:,0] , dff.iloc[:,1] ,  cmap="Blues", shade=True, shade_lowest=False , ax = axes[1])

### Bivariate analysis for Categorical variable

In [ ]:
# Bivariate analysis for categorical variables : Common function to perform correlation within data frame 

def bi_analysis_cat(df):
    sns.set(rc={'figure.figsize':(15,6)})
   
    
    ct = pd.crosstab(df.iloc[:,1] , df.iloc[:,2] , margins = True , \
                     margins_name = 'Total' , normalize = True).round(4) * 100
    
    print('The summary table is as below')
    print(ct)
    print('**********************************************************************************')
       
    sns.violinplot( x = df.iloc[:,1] , y = df.iloc[:,0] ,  hue = df.iloc[:,2] , data = df , \
                   vert = True , inner = 'quartile' ,scale_hue = True , split = True , annot = True )

### Independent t test

The following function is created to perform t test between two numeric distributions. This function will accept two different distributions separately and provide p-value of the test.

In [ ]:
# Function to perform t-test for 2 independent data sets 


def independant_ttest(x , y ):
    stat , p = ss.ttest_ind(x, y , equal_var = False)
    #print('t-test stats value =', round(stat, 2))
    print('p value = ' , p)
    
    if ( p < 0.05):
        print('The 2 sets of data differ significantly')
    else:
        print('We cannot state that the 2 sets of data differ significantly')

### Chi-squared test

The following function is created to perform chi-squared test between two categorical fields. This function will accept data frame having subject column and loan_status column and provide p-value of for independence test.

In [ ]:
# Create a function for a 2 set test using Pearsons's chi2 method


def chi2_cat_pearson(df):
    ctab = pd.crosstab(df.iloc[:,0] , df.iloc[:,1])
    stat , p , dof , expected = ss.chi2_contingency(ctab)
    
    #print('chi2 stats value is =', round(stat,2))
    print('p value =', p)
    
    if ( p < 0.05):
        print('The 2 data sets are not independent of each other ')
    else:
        print('The 2 data sets are independent of each other')

### Tukey's HSD

The Tukey's honestly significant difference test (Tukey's HSD) is used to test differences among sample means for significance. The Tukey's HSD tests all pairwise differences while controlling the probability of making one or more Type I errors.  
This function will accept data frame having subject column and loan_status column and provide test results.

In [ ]:
# Create a function for Tukey's HSD check

def tukeys_hsd(df, p_value=0.05):
    #dta = pd.melt(df, id_vars=list(df.columns)[1], value_vars=list(df.columns)[0])
    #print(dta.head())
    print("Using the pairwise_tukeyhsd Method")
    print("----------------------------------------------------------")
    res2 = pairwise_tukeyhsd(groups= df.iloc[:,1], endog = df.iloc[:,0],  alpha=p_value)
    print("summary:", res2.summary())
    print("mean diffs:", res2.meandiffs)
    print("std pairs:",res2.std_pairs)
    print("groups unique: ", res2.groupsunique)
    print("df total:", res2.df_total)
    p_values = psturng(np.abs(res2.meandiffs / res2.std_pairs), len(res2.groupsunique), res2.df_total)
    print("p values:", p_values)

### Chi-squared post-hoc test

Chi2 Post-hoc test is done post chi2 test to investigate pairwise tests further. This will help in finding out the values in subject categorical variable that are most impacting. This performs Bonferroni adjustment on the p-value based on number of multiple pair test which helps in reducing Type-1 errors.  
This function will accept data frame having subject column and loan_status column and provide values from subject variable which have significant impact on loan status

In [ ]:
def chi2_post_hoc(df, p_value = 0.05):
    # prepare data for multiple pairwise tests
    if df.iloc[:,0].dtypes != 'object':
        dummies = pd.get_dummies(df.iloc[:,0],prefix = 'value')
    else:
        dummies = pd.get_dummies(df.iloc[:,0])
    
    #dummies.drop(list(dummies.columns)[0],axis= 1, inplace= True)
    
    adjusted_p = p_value/dummies.shape[1]
    print("Bonferroni-adjusted p-value: ",adjusted_p)
    for series in dummies:
        nl = "\n"
        
        crosstab = pd.crosstab(dummies[f"{series}"], df.iloc[:,1])
        #print(crosstab, nl)
        chi2, p, dof, expected = ss.chi2_contingency(crosstab)
        if p < adjusted_p:
            print("Pairwise comparisons is significant for:",dummies[f"{series}"].name)
            print(f"p-value= {p}{nl}")   

## Analysis of consumer attributes

### `addr_state`: The state provided by the borrower in the loan application

In [ ]:
uni_analysis_cat(loan_df[['addr_state' , 'loan_status']] )

The mosaic plot shows a lot of variance in fully paid and charged off numbers in different states. NV has the highest proportion (23%) of charged off customers.  
Let's check for the significance in chi2 test

In [ ]:
# chi2 test for addr_state
chi2_cat_pearson(loan_df[['addr_state','loan_status']])

Chi2 test says the variable is significant. 

In [ ]:
# chi2 post-hoc test for addr_state
chi2_post_hoc(loan_df[['addr_state','loan_status']]) 

Post hoc test confirms the claim. However we this this variable cannot be controlled by Lending Club. Therefore will NOT be considered as one of the most significant drivers.

### `annual_inc`: The self-reported annual income provided by the borrower during registration

In [ ]:
uni_analysis_num(loan_df[['annual_inc', 'loan_status']], log_scale=True)

The segmented box plot clearly shows there is variation in median and mean of annual income between Fully paid and charged off segments even in log scale. I.e. in actual scale, the differences will be very significant.  
Lets test for statistical significance in difference using t-test.

In [ ]:
# Independent T test for annual_income
independant_ttest(loan_df[loan_df.loan_status == "Charged Off"].annual_inc ,\
                  loan_df[loan_df.loan_status == "Fully Paid"].annual_inc)

t-test indicates that the annual income distributions are significantly different for fully paid and charged off segments for significant level 0.05.  
We'll now confirm the same using post-hoc test Tukey's HSD with alpha level 0.05 below

In [ ]:
# Tukey's HSD for annual_income
tukeys_hsd(loan_df[['annual_inc','loan_status']])

The above test confirms that we can reject the null hypothesis and from the mean difference we can conclude lower the annual income higher the chances for customer to default 

__`annual_income` is one of the most important variables that Lending club should be interested in to make the decision__

### `verification_status`: Indicates if income was verified by LC, not verified, or if the income source was verified

In [ ]:
uni_analysis_cat(loan_df[['verification_status', 'loan_status']])

The Mosaic plot shows some variation in fully paid and charged off numbers in different levels of verification status.  
Lets test this with chi2 test

In [ ]:
# chi2 test for home_ownership
chi2_cat_pearson(loan_df[['verification_status','loan_status']])

chi2 test shows that verification status has impact on loan_status.

In [ ]:
# chi2 post-hoc test for verification_status
chi2_post_hoc(loan_df[['verification_status','loan_status']])

post-hoc test confirms the claim from chi2 test. However does not make right intuition as verified income has more charged off scenarios.

This highlights possible process gap, may be Lending Club should re-look at there income verification process

### `dti`: A ratio calculated using the borrower’s total monthly debt payments on the total debt obligations, excluding mortgage and the requested LC loan, divided by the borrower’s self-reported monthly income

In [ ]:
uni_analysis_num(loan_df[['dti', 'loan_status']])

Box plot indicates median difference of dti for fully paid and charged off. Mean is also different for these segments but less than median differene.

In [ ]:
# Independent T test for dti
independant_ttest(loan_df[loan_df.loan_status == "Charged Off"].dti ,\
                  loan_df[loan_df.loan_status == "Fully Paid"].dti)

t-test shows that these 2 distributions are not same.

In [ ]:
# Tukey's HSD for dti
tukeys_hsd(loan_df[['dti','loan_status']])

Tukey's HSD confirms the claim. However as we are considering annual income as one of significant variable 

### `emp_length`: Employment length in years

In [ ]:
uni_analysis_cat(loan_df[['emp_length', 'loan_status']])

Mosaic plot does not show a lot of variance between fully paid and charged off of different employment years except where the employment length is not provided. 

In [ ]:
# chi2 test for emp_length
chi2_cat_pearson(loan_df[['emp_length','loan_status']])

Chi2 test showing that these two variable are not independent

In [ ]:
# chi2 post-hoc test for emp_length
chi2_post_hoc(loan_df[['emp_length','loan_status']]) 

post-hoc test confirms the claim. However we don't find this to be very significant specially where employee length is provided. This might mean another process issues. Lending Club should investigate why employee length is not provided. If this is because customer is not employed, then in what grounds loan is getting approved

### `home_ownership`: The home ownership status provided by the borrower during registration

In [ ]:
# Univariate analysis of Home ownership and comparison with home status

uni_analysis_cat(loan_df[['home_ownership' , 'loan_status']])

In [ ]:
# chi2 test for home_ownership

chi2_cat_pearson(loan_df[['home_ownership','loan_status']])

In [ ]:
# chi2 post-hoc test for home_ownership
chi2_post_hoc(loan_df[['home_ownership','loan_status']]) 

Mosaic plot does not show a lot of variance between fully paid and charged off for different home ownership  statuses even if chi2 and post-hoc states otherwise. We are NOT considering this to be a significant variable.

### `mths_since_last_record`: The number of months since the last public record

In [ ]:
uni_analysis_num(loan_df[['mths_since_last_record', 'loan_status']])

In [ ]:
# Independent T test for mths_since_last_record
independant_ttest(loan_df[loan_df.loan_status == "Charged Off"].mths_since_last_record ,\
                  loan_df[loan_df.loan_status == "Fully Paid"].mths_since_last_record)

In [ ]:
# Tukey's HSD for mths_since_last_record
tukeys_hsd(loan_df[['mths_since_last_record','loan_status']])

The spread for fully paid and charged off loans seams to be same, so distinction cannot be made
further, the values are concentrated between 0 and 20.  
Even if T test and Tukey's HSD states otherwise, we are NOT considering this to be a significant variable.


### `open_acc`: The number of open credit lines in the borrower's credit file

In [ ]:
uni_analysis_num(loan_df[['open_acc', 'loan_status']])

In [ ]:
# Independent T test for open_acc
independant_ttest(loan_df[loan_df.loan_status == "Charged Off"].open_acc ,\
                  loan_df[loan_df.loan_status == "Fully Paid"].open_acc)

In [ ]:
# Tukey's HSD for open_acc
tukeys_hsd(loan_df[['open_acc','loan_status']])

The spread for fully paid and charged off loans seams to be same, so distinction cannot be made.  
This is confirmed by both t test and Tukey's HSD test

### `total_acc`: The total number of credit lines currently in the borrower's credit file

In [ ]:
uni_analysis_num(loan_df[['total_acc', 'loan_status']])

In [ ]:
# Independent T test for total_acc
independant_ttest(loan_df[loan_df.loan_status == "Charged Off"].total_acc ,\
                  loan_df[loan_df.loan_status == "Fully Paid"].total_acc)

In [ ]:
# Tukey's HSD for total_acc
tukeys_hsd(loan_df[['total_acc','loan_status']])

The spread for fully paid and charged off loans seams to be same, so distinction cannot be made
further, the values are concentrated between 0 and 20.  
Even if T test and Tukey's HSD states otherwise, we are NOT considering this to be a significant variable.

### `pub_rec`: Number of derogatory public records

Even if the values are Numeric, we are considering this to be categorical as only 4 values are available

In [ ]:
# Doing Univariate analysis 
uni_analysis_cat(loan_df[['pub_rec', 'loan_status']])

Mosaic Plot shows a significant variation between public record 0 and 1 (14% compared to 23% respectively). 

In [ ]:
# chi2 test for pub_rec
chi2_cat_pearson(loan_df[['pub_rec','loan_status']])

Chi2 test also confirms that pub_rec has significant impact on loan status.

In [ ]:
# chi2 post-hoc test for pub_rec
chi2_post_hoc(loan_df[['pub_rec','loan_status']])

post-hoc test confirms that, if the Public record is 1 (or more), there is a high propensity to default

__`pub_rec` is one of the most important variables that Lending club should be interested in to make the decision__

### `pub_rec_bankruptcies`: Number of public record bankruptcies

Even if the values are Numeric, we are considering this to be categorical as only 3 values are available

In [ ]:
# Doing Univariate analysis 
uni_analysis_cat(loan_df[['pub_rec_bankruptcies', 'loan_status']])

Mosaic Plot shows a significant variation between pub_rec_bankruptcies 0 and 1 (14% compared to 22% respectively). 

In [ ]:
# chi2 test for pub_rec_bankruptcies
chi2_cat_pearson(loan_df[['pub_rec_bankruptcies','loan_status']])

In [ ]:
# chi2 post-hoc test for pub_rec_bankruptcies
chi2_post_hoc(loan_df[['pub_rec_bankruptcies','loan_status']])

If the pub_rec_bankruptcies record is 1 or more, there is a high propensity to default

__`pub_rec_bankruptcies` is one of the most important variables that Lending club should be interested in to make the decision__

We'll later check for correlation between `pub_rec` and `pub_rec_bankruptcies` to confirm whether we need to keep both

### `earliest_cr_line`:	The month the borrower's earliest reported credit line was opened

This field is in month-year format. we'll split this month and year to analyze this field

In [ ]:
# In case of months , perform analysis based on both month and year 
# start with the setting the date type and then continue with the seperate analysis of Month and year 

loan_df.earliest_cr_line = pd.to_datetime(loan_df.earliest_cr_line , format = '%b-%y')

loan_df['earliest_cr_line_month'] = loan_df.earliest_cr_line.dt.month
# need to take care of the years as transformation resulting in future years
loan_df['earliest_cr_line_year'] = loan_df.earliest_cr_line.dt.year.apply(lambda x: x-100 if x > 2019 else x)

The value of month is numeric, but has been considered as categorical.

In [ ]:
# Univariate analysis considering the month of the earliest credict line 
uni_analysis_cat(loan_df[['earliest_cr_line_month' , 'loan_status']] , summary = True)

In [ ]:
# chi2 test for earliest_cr_line_month
chi2_cat_pearson(loan_df[['earliest_cr_line_month','loan_status']])

In [ ]:
# chi2 post-hoc test for earliest_cr_line_month
chi2_post_hoc(loan_df[['earliest_cr_line_month','loan_status']])

Mosaic plot does not show a lot of variance between fully paid and charged off for different home ownership statuses.  
Even if chi2 states otherwise, post-hoc test could not identify any particular month as most significant.  
we are NOT considering this to be a significant variable.

In [ ]:
# Univariate analysis considering the year of earliest credit line 
# Here we see that delinquency increases witht the later yeas for growth when loans may have increase just to meet the target numbers

uni_analysis_cat(loan_df[['earliest_cr_line_year' , 'loan_status']])

In [ ]:
# chi2 test for earliest_cr_line_year
chi2_cat_pearson(loan_df[['earliest_cr_line_year','loan_status']])

In [ ]:
# chi2 post-hoc test earliest_cr_line_year
chi2_post_hoc(loan_df[['earliest_cr_line_year','loan_status']])

Mosaic plot shows that proportion of charged off is continusly increasing post 2004.  Both Chi2 and post hoc test confirms that the variable has significant impact on loan status. However Lending Club will not have direct control on this variable.  
However, this can indicate that the process of approving first credit line has deteriorated over due course. 

## Analysis of loan attributes

### `loan_amnt`: The listed amount of the loan applied for by the borrower

In [ ]:
uni_analysis_num(loan_df[['loan_amnt' , 'loan_status']])

In [ ]:
# Independent T test for loan_amnt
independant_ttest(loan_df[loan_df.loan_status == "Charged Off"].loan_amnt ,\
                  loan_df[loan_df.loan_status == "Fully Paid"].loan_amnt)

In [ ]:
# Tukey's HSD for loan_amnt
tukeys_hsd(loan_df[['loan_amnt' , 'loan_status']])

The spread for fully paid and charged off loans seams to be same, so distinction cannot be made.  
Even if T test and Tukey's HSD states otherwise, we are NOT considering this to be a significant variable.

### `installment`: The monthly payment owed by the borrower if the loan originates

In [ ]:
uni_analysis_num(loan_df[['installment' , 'loan_status']])

In [ ]:
# Independent T test for installment
independant_ttest(loan_df[loan_df.loan_status == "Charged Off"].installment ,\
                  loan_df[loan_df.loan_status == "Fully Paid"].installment)

In [ ]:
# Tukey's HSD for installment
tukeys_hsd(loan_df[['installment','loan_status']])

The spread for fully paid and charged off loans seams to be same, so distinction cannot be made.  
Even if T test and Tukey's HSD states otherwise, we are NOT considering this to be a significant variable.

### `int_rate`: Interest Rate on the loan

In [ ]:
uni_analysis_num(loan_df[['int_rate' , 'loan_status']])

In [ ]:
# Independent T test for int_rate
independant_ttest(loan_df[loan_df.loan_status == "Charged Off"].int_rate ,\
                  loan_df[loan_df.loan_status == "Fully Paid"].int_rate)

In [ ]:
# Tukey's HSD for int_rate
tukeys_hsd(loan_df[['int_rate','loan_status']])

The median and distribution for fully paid and charged off are significantly different. T-test and Tukey's HSD confirms the claim.

__`int_rate` is one of the most important indicators that Lending club should be interested in. However it is decided based on grade and sub-grade. Hence final significance should be based on significance of grade / sub_grade__

### `term`: The number of payments on the loan

term is a numeric value but has been considered as categorical as it has only 2 possible values.

In [ ]:
uni_analysis_cat(loan_df[['term' , 'loan_status']])

In [ ]:
# chi2 test for term
chi2_cat_pearson(loan_df[['term','loan_status']])

In [ ]:
# chi2 post-hoc test for term
chi2_post_hoc(loan_df[['term','loan_status']]) 

The mosaic plot show significant difference in proportion for fully paid and charged off numbers for different terms.  
Both ci2 test and post-hoc test indicates the same.

__`term` is one of the most important variables that Lending club should be interested in to make the decision__

### `funded_amnt`: The total amount committed to that loan at that point in time

In [ ]:
uni_analysis_num(loan_df[['funded_amnt', 'loan_status']])

In [ ]:
# Independent T test for funded_amnt
independant_ttest(loan_df[loan_df.loan_status == "Charged Off"].funded_amnt ,\
                  loan_df[loan_df.loan_status == "Fully Paid"].funded_amnt)

In [ ]:
# Tukey's HSD for funded_amnt
tukeys_hsd(loan_df[['funded_amnt','loan_status']])

The spread for fully paid and charged off loans seams to be very similar, so distinction is difficult.
Even if T test and Tukey's HSD indicates significant difference in distribution, we are NOT sure regarding the nature of the variable, whether is post loan or pre-loan. We'll take decision on this variable during bi variate analysis.

### `funded_amnt_inv`: The total amount committed by investors for that loan at that point in time

In [ ]:
uni_analysis_num(loan_df[['funded_amnt_inv', 'loan_status']])

In [ ]:
# Independent T test for annual_income
independant_ttest(loan_df[loan_df.loan_status == "Charged Off"].funded_amnt_inv ,\
                  loan_df[loan_df.loan_status == "Fully Paid"].funded_amnt_inv)

In [ ]:
# Tukey's HSD for funded_amnt_inv
tukeys_hsd(loan_df[['funded_amnt_inv','loan_status']])

The spread for fully paid and charged off loans seams to be very similar, so distinction is difficult.
Even if T test and Tukey's HSD indicates significant difference in distribution, we are NOT sure regarding the nature of the variable, whether is post loan or pre-loan. We'll take decision on this variable during bi variate analysis.

### `grade`: LC assigned loan grade

In [ ]:
uni_analysis_cat(loan_df[['grade', 'loan_status']])

In [ ]:
# chi2 test for grade
chi2_cat_pearson(loan_df[['grade','loan_status']])

In [ ]:
# chi2 post-hoc test for grade
chi2_post_hoc(loan_df[['grade','loan_status']])

The mosaic plot show significant difference in proportion for fully paid and charged off numbers for different terms.  
Both ci2 test and post-hoc test indicates the same. Post-hoc test identified all values of grade to be very significant.  

However int_rate is decided based on grade and sub grade. - https://www.lendingclub.com/foliofn/rateDetail.action  
We shall consider grade and sub-grade as significant variables and not include interest rate which is derived from the same

### `sub_grade`: LC assigned loan subgrade

In [ ]:
uni_analysis_cat(loan_df[['sub_grade' , 'loan_status']] , summary = True)

In [ ]:
# chi2 test for sub_grade
chi2_cat_pearson(loan_df[['sub_grade','loan_status']])

In [ ]:
# chi2 post-hoc test for sub_grade
chi2_post_hoc(loan_df[['sub_grade','loan_status']])

The mosaic plot show significant difference in proportion for fully paid and charged off numbers for different terms.  
Both ci2 test and post-hoc test indicates the same. Post-hoc test identified a number of values of sub-grade to be very significant.  

However int_rate is decided based on grade and sub grade. - https://www.lendingclub.com/foliofn/rateDetail.action  
We shall consider grade and sub-grade as significant variables and not include interest rate which is derived from the same

### `purpose`: A category provided by the borrower for the loan request

In [ ]:
uni_analysis_cat(loan_df[['purpose' , 'loan_status']] , summary = True)

In [ ]:
# chi2 test for home_ownership
chi2_cat_pearson(loan_df[['purpose','loan_status']])

In [ ]:
# chi2 post-hoc test for purpose
chi2_post_hoc(loan_df[['purpose','loan_status']])

The mosaic plot show significant difference in proportion for fully paid and charged off numbers for different terms.  
Both ci2 test and post-hoc test indicates the same.   
We can see that small_business, renewable_energy, educational are purposes where the propensity for default is high.

__`purpose` is an important parameter as it has relevance for Default, we'll consider it as one of the most important variable that Lending Club will be interested in decision making__

### `revol_bal`: Total credit revolving balance

In [ ]:
# Doing Univariate analysis 
uni_analysis_num(loan_df[['revol_bal', 'loan_status']])

In [ ]:
# Independent T test for revol_bal
independant_ttest(loan_df[loan_df.loan_status == "Charged Off"].revol_bal ,\
                  loan_df[loan_df.loan_status == "Fully Paid"].revol_bal)

In [ ]:
# Tukey's HSD for revol_bal
tukeys_hsd(loan_df[['revol_bal' , 'loan_status']])

The spread for fully paid and charged off loans seams to be same, so distinction cannot be made. T-test and Tukeu's HSD support the claim.

### `revol_util`: Revolving line utilization rate, or the amount of credit the borrower is using relative to all available revolving credit

In [ ]:
uni_analysis_num(loan_df[['revol_util' , 'loan_status']])


In [ ]:
# Independent T test for revol_util
independant_ttest(loan_df[loan_df.loan_status == "Charged Off"].revol_util ,\
                  loan_df[loan_df.loan_status == "Fully Paid"].revol_util)

In [ ]:
# Tukey's HSD for revol_util
tukeys_hsd(loan_df[['revol_util' , 'loan_status']])

The mean, median and distribution is significantly different between fully paid and charged off.  
The above test confirms that we can reject the null hypothesis and from the mean difference we can conclude higher the revolving debt utilization higher the chances for customer to default 

__`revol_util` is one of the most important variables that Lending club should be interested in to make the decision__

### `issue_d`: The month which the loan was funded

This is a date variable in month-year form. This will be first split in month and year for further analysis.

In [ ]:
# In case of months , perform analysis based on both month and year 
# start with the setting the date type and then continue with the seperate analysis of Month and year 

loan_df.issue_d = pd.to_datetime(loan_df.issue_d , format = '%b-%y')

loan_df['issue_d_month'] = loan_df.issue_d.dt.month
loan_df['issue_d_year'] = loan_df.issue_d.dt.year

#Compress the data for the issue date to display the same in the consolidated form based on % 

df_issue_d = loan_df.loc[loan_df.loan_status == 'Charged Off' , ['issue_d_year' , 'issue_d_month' , 'loan_status']]\
            .groupby(['issue_d_year' , 'issue_d_month' , 'loan_status']).size().reset_index()

df_issue_d['status_percent'] = (df_issue_d[0]/sum(df_issue_d[0])).round(4)*100


month is a numeric values but treated as ordinal.

In [ ]:
# Univariate analysis considering the month of the loan issue 
# Here we see that delinquency increases witht the later months when loans may have increase just to meet the target numbers

uni_analysis_cat(loan_df[['issue_d_month' , 'loan_status']] , summary = True)

In [ ]:
# chi2 test for issue_d_month
chi2_cat_pearson(loan_df[['issue_d_month','loan_status']])

In [ ]:
# chi2 post-hoc test for issue_d_month
chi2_post_hoc(loan_df[['issue_d_month','loan_status']]) 

Mosaic plot does not show a lot of variance between fully paid and charged off for different issue month.  
Even if chi2 and post-hoc states otherwise, we are NOT considering this to be a significant variable.

In [ ]:
# Univariate analysis considering the year of the loan issue 
uni_analysis_cat(loan_df[['issue_d_year' , 'loan_status']])

In [ ]:
# chi2 test for issue_d_year
chi2_cat_pearson(loan_df[['issue_d_year','loan_status']])

In [ ]:
# chi2 post-hoc test for issue_d_year
chi2_post_hoc(loan_df[['issue_d_year','loan_status']]) 

Mosaic plot shows that proportion of charged off is higher in 2007, 2008 and 2011.  Both Chi2 and post hoc test confirms that the variable has significant impact on loan status. However Lending Club will not have direct control on this variable.  
However, this can indicate that the process of issuing loan is not always efficient. 

In [ ]:
# Undertake the scatter plot for the charged Off variables

bubble_plot(df_issue_d.issue_d_month , df_issue_d.issue_d_year , df_issue_d.loan_status , df_issue_d.status_percent)

Here we see that delinquency increases witht the later yeas for growth when loans may have increase just to meet the target numbers.

### `inq_last_6mths`: The number of inquiries in past 6 months (excluding auto and mortgage inquiries)

This value is numeric, but treated as categorical as there are only 8 values.

In [ ]:
uni_analysis_cat(loan_df[['inq_last_6mths' , 'loan_status']])

In [ ]:
# chi2 test for inq_last_6mths
chi2_cat_pearson(loan_df[['inq_last_6mths','loan_status']])

In [ ]:
# chi2 post-hoc test for inq_last_6mths
chi2_post_hoc(loan_df[['inq_last_6mths','loan_status']]) 

The mosaic plot show significant difference in proportion for fully paid and charged off numbers for different levels of inquiries.  

Both ci2 test and post-hoc test indicates the same. post hoc test indicates value greater that risk has potential risk for delinquency. This can be used by Lending Club to assess risk of delinquency.

  As not a lot of customers may do an enquiry with Lending Club, this data may not be always availible completely to make a decision

## Derived Metrics

### Monthly income - binned 

The purpose of the derived variable is create bins for monthly income can visualize the same as categorical variable and the establish the same claim that annual income is one of the most significant variables

In [ ]:
# annual income to monthly income
loan_df['monthly_inc'] =  loan_df.annual_inc/12
loan_df['monthly_inc'].describe()

In [ ]:
# bins to be created
bins = [300, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000, 50000, 500000]
# bin labels
labels = ['300-1000', 
          '1000-2000', 
          '2000-3000', 
          '3000-4000', 
          '4000-5000', 
          '5000-6000', 
          '6000-7000', 
          '7000-8000', 
          '8000-9000', 
          '9000-10000', 
          '10000-50000', 
          '50000-500000']
loan_df['monthly_inc_bins'] =  pd.cut(loan_df.monthly_inc, bins = bins, labels=labels)
uni_analysis_cat(loan_df[['monthly_inc_bins', 'loan_status']])

In [ ]:
# chi2 test for monthly_inc_bins
chi2_cat_pearson(loan_df[['monthly_inc_bins','loan_status']])

In [ ]:
# chi2 post-hoc test for issue_d_year
chi2_post_hoc(loan_df[['monthly_inc_bins','loan_status']]) 

mosaic plots, chi2 test and post hoc test confirms that monthly income bins and in turn annual income as significant impact of delinquency. There is no need to select this variable as this derived from annual income straightway.

### Loan Amount as Percentage of Annual Income

In [ ]:
# % of loan amount with respect to annual inc
loan_df['loan_amount_by_annual_inc'] = loan_df.loan_amnt/loan_df.annual_inc*100
uni_analysis_num(loan_df[['loan_amount_by_annual_inc', 'loan_status']])

In [ ]:
# Independent T test for loan_amount_by_annual_inc
independant_ttest(loan_df[loan_df.loan_status == "Charged Off"].loan_amount_by_annual_inc ,\
                  loan_df[loan_df.loan_status == "Fully Paid"].loan_amount_by_annual_inc)

In [ ]:
# Tukey's HSD for loan_amount_by_annual_inc
tukeys_hsd(loan_df[['loan_amount_by_annual_inc','loan_status']])

The mean, median and distribution is significantly different between fully paid and charged off.  
The above test confirms that we can reject the null hypothesis and from the mean difference we can conclude higher the ratio between loan_amount and annual_inc higher the chances for customer to default 

__`loan_amount_by_annual_inc` is one of the most important variables that Lending club should be interested in to make the decision__

### Annual Installment as Percentage of Annual Income

In [ ]:
# % of loan installment with respect to monthly income
loan_df['annual_installment_by_annual_inc'] = ((loan_df.installment * 12)/loan_df.annual_inc)*100
uni_analysis_num(loan_df[['annual_installment_by_annual_inc', 'loan_status']])

In [ ]:
# Independent T test for annual_installment_by_annual_inc
independant_ttest(loan_df[loan_df.loan_status == "Charged Off"].annual_installment_by_annual_inc ,\
                  loan_df[loan_df.loan_status == "Fully Paid"].annual_installment_by_annual_inc)

In [ ]:
# Tukey's HSD for annual_installment_by_annual_inc
tukeys_hsd(loan_df[['annual_installment_by_annual_inc','loan_status']])

The mean, median and distribution is significantly different between fully paid and charged off.  
The above test confirms that we can reject the null hypothesis and from the mean difference we can conclude higher the ratio between annual installment and annual_inc higher the chances for customer to default 

__`annual_installment_by_annual_inc` is one of the most important variables that Lending club should be interested in to make the decision__

### Surplus amount per month without considering current loan installment

In [ ]:
# Surplus amount per month without considering current loan installment
loan_df['surplus_amount_per_month'] = (1 - (loan_df.dti/100)) * loan_df.annual_inc/12
uni_analysis_num(loan_df[['surplus_amount_per_month', 'loan_status']], log_scale=True)

In [ ]:
# Independent T test for surplus_amount_per_month
independant_ttest(loan_df[loan_df.loan_status == "Charged Off"].surplus_amount_per_month ,\
                  loan_df[loan_df.loan_status == "Fully Paid"].surplus_amount_per_month)

In [ ]:
# Tukey's HSD for surplus_amount_per_month
tukeys_hsd(loan_df[['surplus_amount_per_month','loan_status']])

The mean, median and distribution is significantly different between fully paid and charged off.  
The above test confirms that we can reject the null hypothesis and from the mean difference we can conclude lesser the surplus amount higher the chances for customer to default 

__`surplus_amount_per_month` is one of the most important variables that Lending club should be interested in to make the decision__

### Funded amount as proportion to Loan amount

In [ ]:
loan_df['funded_amnt_by_loan_amnt'] = loan_df.funded_amnt/loan_df.loan_amnt
uni_analysis_num(loan_df[['funded_amnt_by_loan_amnt', 'loan_status']])

In [ ]:
# Independent T test for funded_amnt_by_loan_amnt
independant_ttest(loan_df[loan_df.loan_status == "Charged Off"].funded_amnt_by_loan_amnt ,\
                  loan_df[loan_df.loan_status == "Fully Paid"].funded_amnt_by_loan_amnt)

In [ ]:
# Tukey's HSD for funded_amnt_by_loan_amnt
tukeys_hsd(loan_df[['funded_amnt_by_loan_amnt','loan_status']])

The spread for fully paid and charged off loans seams to be same, distinction is difficult.  
Even if T test and Tukey's HSD states otherwise, we are NOT considering this to be a significant variable.

### Open_acc in proportion of total_acc

In [ ]:
# derived matrix based on open_acc and total_acc
loan_df['open_acc_by_total_acc'] = loan_df.open_acc/loan_df.total_acc

In [ ]:
uni_analysis_num(loan_df[['open_acc_by_total_acc', 'loan_status']])

In [ ]:
# Independent T test for open_acc_by_total_acc
independant_ttest(loan_df[loan_df.loan_status == "Charged Off"].open_acc_by_total_acc ,\
                  loan_df[loan_df.loan_status == "Fully Paid"].open_acc_by_total_acc)

In [ ]:
# Tukey's HSD for open_acc_by_total_acc
tukeys_hsd(loan_df[['open_acc_by_total_acc','loan_status']])

The derived matrix open_acc_by_total_acc suggest nothing in particular, as distribution is similar to open_acc 

## Bivariate Analysis

### Correlation Analysis

We'll now perform correlation analysis on all numeric values including the derived metrics. This will also include numeric variables that have been considered as categorical for univariate analysis

In [ ]:
# this is correlation plot for all numeric variables
sns.heatmap(loan_df.corr(), cmap = sns.cm.rocket_r)

In [ ]:
loan_df.corr().style.background_gradient()

It is difficult to read the complete matrix, so we'll create smaller correlation matrix with only the variables which have correlation value more than 0.5 with at least another variable.

In [ ]:
# find columns where they have absolute correlation greater than 0.5 with some other column
corr_matrix = loan_df.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
to_keep_col = [column for column in upper.columns if any(upper.loc[:,column] >=0.5)]
to_keep_row = [row for row in upper.index if any(upper.loc[row,:] >=0.5)]
to_keep = list(set(to_keep_col).union(set(to_keep_row)))
to_keep

Lets Focus on these highly correlated columns to derive new features.

In [ ]:
#subset dataframe on highly correlated columns
loan_high_corr_df = loan_df.loc[:,to_keep]
sns.heatmap(loan_high_corr_df.corr(),cmap =sns.cm.rocket_r)

In [ ]:
loan_high_corr_df.corr().style.background_gradient()

This gives us following list of highly correlated variables from our earlier identified most important variables.
* `annual_inc` - `surplus_amount_per_month`: consider any of these in important list of variables
* `loan_amount_by_annual_inc` - `annual_installment_by_annual_inc`: consider any of these in important list of variables
* `pub_rec_bankruptcies` - `pub_rec`: consider any of these in important list of variables


### Int_rate vs DTI

In [ ]:
# Finding the correlation between the interest rate and the dti for loans which are paid and charged off

bi_analysis_num(loan_df[['int_rate' , 'dti' , 'loan_status']])

From above we observe that at same level of dti , if the loan has a higher interest rate ,there are high chances of default 

### DTI vs purpose

In [ ]:
# Bivariate analysis of dti and purpose

bi_analysis_cat(loan_df[['dti' , 'purpose' , 'loan_status']])

The purpose for which loan is applied does indicate some chances for default. Some of the purposes have been identified as more risky ventures as compared to others . Analysing purpose alongwith dti, we see that loans taken for car , small business , home improvement are risky even at lower dti

### Dti vs home ownership 

In [ ]:
bi_analysis_cat(loan_df[['dti' , 'home_ownership' , 'loan_status']])

The type of home ownership do not show any significant difference on the final loan status. 

### dt vs inquiries in the last 6 months to understanding the availablity background information 

In [ ]:
# Understanding the relation between dti and enquiries in last 6 months 

print(loan_df[['dti' , 'inq_last_6mths']].corr())

bi_analysis_cat(loan_df[['dti' , 'inq_last_6mths' , 'loan_status']])

As number of inquiries for loan increase the possibility to default at lower dti increases . Thus number of inquiries is a good leading indicator for the tendency to borrow and default even with lower dtis. However since the correlation between inquiries and dti is low , it does not indicate a 'credit seeking tendency'

### Funded amount invested vs past bankruptcies 

In [ ]:
print(loan_df[['funded_amnt_inv' , 'inq_last_6mths']].corr())

print('**************************************************************************************************')

bi_analysis_cat(loan_df[['funded_amnt_inv' , 'pub_rec_bankruptcies' , 'loan_status']])

The correlation between funded amount invested and number of bankrupties is very low. Thus the members who invest in loan do not have enough information about the past bankrupties. 

Past bankruptcy is an established indicator of default . Transfer of the infromation to the investors can reduce the investment in risky loans 

### Funded amount vs dti

In [ ]:
bi_analysis_num(loan_df[['funded_amnt_inv' , 'dti' , 'loan_status']])

It seems that Information of the dti not available to the investors since higher dti should ideally reduce the investment in the loan and thus serve as a leadfing indicator 

## Conclusions

Our analysis concludes that following are the most important variables that Lending Club should be interested in while making decision regarding loan approval.
* Customer Variable
    1. `annual_inc`
    2. `pub_rec` / `pub_rec_bankruptcies`
* Loan Variable
    3. `grade` / `sub_grade` (We are not considering int_rate anymore)
    4. `term`
    5. `revol_utl`
* Derived variable
    6. `annual_installment_by_annual_inc` / `loan_amount_by_annual_inc`
    7. `surplus_amount_per_month`